In [1]:
import tensorflow as tf

In [61]:
import math
import numpy as np
import pandas as pd
import random
import statistics
from collections import Counter
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredLogarithmicError

In [3]:
train1_data = pd.read_csv('X_train1_oh.csv')
train2_data = pd.read_csv('X_train2_oh.csv')

test_data = pd.read_csv('X_test_oh.csv')


In [4]:
train1_data=train1_data.iloc[:, 1:]

In [5]:
train1_data.shape

(159, 218)

In [6]:
train2_data=train2_data.iloc[:, 1:]

In [7]:
train2_data.shape

(159, 218)

In [8]:
test_data=test_data.iloc[:, 1:]

In [9]:
test_data.shape

(80, 218)

In [10]:
from sklearn.preprocessing import MinMaxScaler

mm_scaler = MinMaxScaler()

x_train1_scaled = mm_scaler.fit_transform(train1_data)
x_train2_scaled = mm_scaler.fit_transform(train2_data)

x_test_scaled = mm_scaler.transform(test_data)

In [11]:
class AutoEncoders(Model):

  def __init__(self, output_units):

    super().__init__()
    print(output_units)
    self.encoder = Sequential(
        [
          Dense(512, activation="relu",kernel_regularizer='l1'),
          #Reducing the dimensionality of features from 218 to 32
          Dense(32, activation="relu")
        ]
    )

    self.decoder = Sequential(
        [
          Dense(256, activation="relu"),
          Dense(output_units, activation="softmax")
        ]
    )

  def call(self, inputs):

    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded
  
auto_encoder = AutoEncoders(len(x_train1_scaled[0]))

auto_encoder.compile(
    loss='mean_squared_logarithmic_error',
    metrics=['mae', 'acc'],
    optimizer='adam'
)

history = auto_encoder.fit(
    x_train1_scaled, 
    x_train1_scaled, 
    epochs=50, 
    batch_size=10,
    #validation_data=(x_test_scaled, x_test_scaled)

)


218
Epoch 1/50
16/16 [==============================] - 1s 8ms/step - loss: 42.9187 - mae: 0.2133 - acc: 0.0000e+00
Epoch 2/50
16/16 [==============================] - 0s 9ms/step - loss: 28.3472 - mae: 0.2101 - acc: 0.0000e+00
Epoch 3/50
16/16 [==============================] - 0s 7ms/step - loss: 16.8461 - mae: 0.2072 - acc: 0.0000e+00
Epoch 4/50
16/16 [==============================] - 0s 8ms/step - loss: 8.4221 - mae: 0.2066 - acc: 0.0000e+00
Epoch 5/50
16/16 [==============================] - 0s 7ms/step - loss: 3.1019 - mae: 0.2066 - acc: 0.0000e+00
Epoch 6/50
16/16 [==============================] - 0s 8ms/step - loss: 0.8929 - mae: 0.2065 - acc: 0.0000e+00
Epoch 7/50
16/16 [==============================] - 0s 7ms/step - loss: 0.4452 - mae: 0.2065 - acc: 0.0000e+00
Epoch 8/50
16/16 [==============================] - 0s 8ms/step - loss: 0.2917 - mae: 0.2065 - acc: 0.0000e+00
Epoch 9/50
16/16 [==============================] - 0s 9ms/step - loss: 0.2510 - mae: 0.2065 - acc: 0.000

In [12]:
x_train1_scaled[0]

array([0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0.])

In [13]:
auto_encoder.summary()


Model: "auto_encoders"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 32)                128544    
                                                                 
 sequential_1 (Sequential)   (None, 218)               64474     
                                                                 
Total params: 193,018
Trainable params: 193,018
Non-trainable params: 0
_________________________________________________________________


In [14]:
f2=auto_encoder.predict(x_train2_scaled)
f3=auto_encoder.predict(x_test_scaled)


In [15]:
encoder_layer = auto_encoder.get_layer('sequential')
f2 = pd.DataFrame(encoder_layer.predict(x_train2_scaled))
f2 = f2.add_prefix('feature_')

In [16]:
#TODO Check if it is sequential_1
encoder_layer = auto_encoder.get_layer('sequential')
f3 = pd.DataFrame(encoder_layer.predict(x_test_scaled))
f3 = f3.add_prefix('feature_')

In [17]:
len(f2)

159

In [18]:
f2

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31
0,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
1,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
2,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
3,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
4,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
155,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
156,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
157,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0


In [19]:
len(f3)

80

In [20]:
f3

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31
0,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
1,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
2,0.904765,0.483633,0.144929,1.294306,1.058510,1.054761,0.293048,0.0,0.0,0.0,...,0.526149,0.848122,0.0,0.651922,0.907061,1.200214,0.0,0.851596,0.0,0.0
3,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
4,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.905558,0.483243,0.144875,1.292970,1.058159,1.054182,0.292297,0.0,0.0,0.0,...,0.526024,0.848651,0.0,0.651302,0.907454,1.199782,0.0,0.850719,0.0,0.0
76,0.905558,0.483243,0.144875,1.292970,1.058159,1.054182,0.292297,0.0,0.0,0.0,...,0.526024,0.848651,0.0,0.651302,0.907454,1.199782,0.0,0.850719,0.0,0.0
77,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
78,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0


In [21]:
from sklearn.ensemble import IsolationForest
if_model=IsolationForest(n_estimators=100, contamination=0.03)
if_model.fit(f2)
print(if_model)

score_pred = if_model.decision_function(f2)
#print(score_pred)
pred = if_model.predict(f3)
#print(pred)
print(type(pred))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


IsolationForest(contamination=0.03)
<class 'numpy.ndarray'>


In [22]:
arr = np.array([pred])
list1 = arr.tolist()
print(list1)
print(type(list1))

Counter(pred)

[[1, 1, -1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1]]
<class 'list'>


Counter({1: 61, -1: 19})

In [23]:
#Second part is to train the positive samples to the same autoencoder architecture

In [24]:
f3['label'] = pred

In [25]:
f3.label

0     1
1     1
2    -1
3     1
4     1
     ..
75   -1
76   -1
77    1
78    1
79    1
Name: label, Length: 80, dtype: int64

In [26]:
f3.shape

(80, 33)

In [27]:
f3.head(5)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,label
0,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1
1,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1
2,0.904765,0.483633,0.144929,1.294306,1.058510,1.054761,0.293048,0.0,0.0,0.0,...,0.848122,0.0,0.651922,0.907061,1.200214,0.0,0.851596,0.0,0.0,-1
3,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1
4,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1


In [28]:
#has only positive logs
p1 = f3[f3.label == 1]

In [29]:
p1.shape

(61, 33)

In [30]:
p1.head(5)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,label
0,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1
1,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1
3,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1
4,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1
5,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1


In [31]:
#has only negative logs
p2 = f3[f3.label == -1]

In [32]:
p2.shape

(19, 33)

In [33]:
split_len = 0.7 * len(p1)
split_len

42.699999999999996

In [34]:
auto2 = p1[:int(split_len)]
autotest = p1[int(split_len):]

In [35]:
#This is the training set and has only positive logs
auto2.shape

(42, 33)

In [36]:
#To reset the index
trainp = auto2.reset_index(drop=True)

In [37]:
trainp.head(5)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,label
0,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1
1,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1
2,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1
3,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1
4,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1


In [38]:
trainp = trainp.drop(['label'], axis=1)

In [39]:
trainp.shape

(42, 32)

In [40]:
frames = [autotest, p2]

In [41]:
autoentest = pd.concat(frames)

In [42]:
testp = autoentest.reset_index(drop=True)

In [43]:
#Has some positive logs and some negative logs and is the testing set
testp.shape

(38, 33)

In [44]:
testp.head(5)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,label
0,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1
1,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1
2,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1
3,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1
4,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,1


In [45]:
testp = testp.drop(['label'], axis = 1)

In [46]:
testp.shape

(38, 32)

In [47]:
len(trainp.iloc[0])

32

In [48]:
#This is the second autoencoder to train on the predicted positive and negative logs.
#This has the same architecture as the first autoencoder

In [49]:
class AutoEncoders(Model):

  def __init__(self, output_units):

    super().__init__()
    print(output_units)
    self.encoder = Sequential(
        [
          Dense(512, activation="relu",kernel_regularizer='l1'),
          #Reducing the dimensionality of features from 218 to 32
          Dense(32, activation="relu")
        ]
    )

    self.decoder = Sequential(
        [
          Dense(256, activation="relu"),
          Dense(output_units, activation="softmax")
        ]
    )

  def call(self, inputs):

    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded
  
auto_encoder2 = AutoEncoders(len(trainp.iloc[0]))

auto_encoder2.compile(
    loss='mean_squared_logarithmic_error',
    metrics=['mae', 'acc'],
    optimizer='adam'
)

#Must pass the training set as both input and output 
#as the encoder will learn to compress the dataset 
history = auto_encoder2.fit(
    trainp, 
    trainp, 
    epochs=10, 
    batch_size=10,
    #validation_data=(x_test_scaled, x_test_scaled)

)


32
Epoch 1/10
5/5 [==============================] - 1s 4ms/step - loss: 8.5740 - mae: 0.5768 - acc: 0.0000e+00
Epoch 2/10
5/5 [==============================] - 0s 6ms/step - loss: 7.7926 - mae: 0.5744 - acc: 0.0000e+00
Epoch 3/10
5/5 [==============================] - 0s 4ms/step - loss: 7.0507 - mae: 0.5703 - acc: 0.0000e+00
Epoch 4/10
5/5 [==============================] - 0s 4ms/step - loss: 6.3493 - mae: 0.5649 - acc: 0.0000e+00
Epoch 5/10
5/5 [==============================] - 0s 4ms/step - loss: 5.6887 - mae: 0.5602 - acc: 0.0000e+00
Epoch 6/10
5/5 [==============================] - 0s 4ms/step - loss: 5.0642 - mae: 0.5576 - acc: 0.0000e+00
Epoch 7/10
5/5 [==============================] - 0s 4ms/step - loss: 4.4793 - mae: 0.5566 - acc: 0.0000e+00
Epoch 8/10
5/5 [==============================] - 0s 4ms/step - loss: 3.9317 - mae: 0.5562 - acc: 0.0000e+00
Epoch 9/10
5/5 [==============================] - 0s 6ms/step - loss: 3.4227 - mae: 0.5561 - acc: 0.0000e+00
Epoch 10/10
5/5 

In [50]:
auto_encoder2.summary()

Model: "auto_encoders_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 32)                33312     
                                                                 
 sequential_3 (Sequential)   (None, 32)                16672     
                                                                 
Total params: 49,984
Trainable params: 49,984
Non-trainable params: 0
_________________________________________________________________


In [54]:
encoder_layer = auto_encoder2.get_layer('sequential_2')
o1 = pd.DataFrame(encoder_layer.predict(trainp))
o1 = trainp.add_prefix('new_')

In [55]:
o1.shape

(42, 32)

In [57]:
o1.head(5)

,new_feature_0,new_feature_1,new_feature_2,new_feature_3,new_feature_4,new_feature_5,new_feature_6,new_feature_7,new_feature_8,new_feature_9,...,new_feature_22,new_feature_23,new_feature_24,new_feature_25,new_feature_26,new_feature_27,new_feature_28,new_feature_29,new_feature_30,new_feature_31
0,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
1,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
2,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
3,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
4,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0


In [69]:
#Compute average and standard deviation of feature values of each sample
o1['o1_mean'] = o1.mean(axis = 1) 
o1['std'] = o1.std(axis = 1)
o1_std = o1['std'] = o1.std(axis = 1)

In [70]:
o1.head(5)

,new_feature_0,new_feature_1,new_feature_2,new_feature_3,new_feature_4,new_feature_5,new_feature_6,new_feature_7,new_feature_8,new_feature_9,...,new_feature_24,new_feature_25,new_feature_26,new_feature_27,new_feature_28,new_feature_29,new_feature_30,new_feature_31,o1_mean,std
0,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.584563,0.49219
1,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.584563,0.49219
2,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.584563,0.49219
3,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.584563,0.49219
4,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.584563,0.49219


In [77]:
#Calculating the threshold is done by multiplying std with a constant c.
#The c is chosen based on the predicted percentage of negative logs in the prediction using IF
threshold_value = o1_std * 0.4

In [78]:
print(threshold_value)

0     0.196876
1     0.196876
2     0.196876
3     0.196876
4     0.196876
5     0.196876
6     0.196876
7     0.196876
8     0.196876
9     0.196876
10    0.196876
11    0.196876
12    0.196876
13    0.196876
14    0.196876
15    0.196876
16    0.196876
17    0.196876
18    0.196876
19    0.196876
20    0.196876
21    0.196876
22    0.196876
23    0.196876
24    0.196876
25    0.196876
26    0.196876
27    0.196876
28    0.196876
29    0.196876
30    0.196876
31    0.196876
32    0.196876
33    0.196876
34    0.196876
35    0.196876
36    0.196876
37    0.196876
38    0.196876
39    0.196876
40    0.196876
41    0.196876
dtype: float32


In [58]:
encoder_layer = auto_encoder2.get_layer('sequential_3')
o2 = pd.DataFrame(encoder_layer.predict(testp))
o2 = testp.add_prefix('new_')

In [59]:
o2.shape

(38, 32)

In [60]:
o2.head(5)

,new_feature_0,new_feature_1,new_feature_2,new_feature_3,new_feature_4,new_feature_5,new_feature_6,new_feature_7,new_feature_8,new_feature_9,...,new_feature_22,new_feature_23,new_feature_24,new_feature_25,new_feature_26,new_feature_27,new_feature_28,new_feature_29,new_feature_30,new_feature_31
0,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
1,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
2,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
3,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0
4,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.524937,0.846731,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0


In [81]:
o2['o2_mean'] = o2.mean(axis = 1) 

In [82]:
o2.head(5)

,new_feature_0,new_feature_1,new_feature_2,new_feature_3,new_feature_4,new_feature_5,new_feature_6,new_feature_7,new_feature_8,new_feature_9,...,new_feature_24,new_feature_25,new_feature_26,new_feature_27,new_feature_28,new_feature_29,new_feature_30,new_feature_31,o1_mean,o2_mean
0,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.587139,0.587139
1,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.587139,0.587139
2,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.587139,0.587139
3,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.587139,0.587139
4,0.90452,0.483407,0.14383,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.0,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.587139,0.587139


In [84]:
#If the mean of the o2 dataset is less than the threshold then that row is anamoly
o2.loc[o2['o2_mean'] < 0.196876, 'label'] = 'Anamoly'  
o2.loc[o2['o2_mean'] > 0.196876, 'label'] = 'not_Anamoly'  
 

In [85]:
o2

,new_feature_0,new_feature_1,new_feature_2,new_feature_3,new_feature_4,new_feature_5,new_feature_6,new_feature_7,new_feature_8,new_feature_9,...,new_feature_25,new_feature_26,new_feature_27,new_feature_28,new_feature_29,new_feature_30,new_feature_31,o1_mean,o2_mean,label
0,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.587139,0.587139,not_Anamoly
1,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.587139,0.587139,not_Anamoly
2,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.587139,0.587139,not_Anamoly
3,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.587139,0.587139,not_Anamoly
4,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.587139,0.587139,not_Anamoly
5,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.587139,0.587139,not_Anamoly
6,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.587139,0.587139,not_Anamoly
7,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.587139,0.587139,not_Anamoly
8,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.587139,0.587139,not_Anamoly
9,0.904520,0.483407,0.143830,1.292297,1.060116,1.055643,0.292808,0.0,0.0,0.0,...,0.649309,0.908203,1.200196,0.0,0.853912,0.0,0.0,0.587139,0.587139,not_Anamoly
